# LangChain PDF Frage-Antwort Beispiel mit SentenceTransformerEmbeddings

In diesem Notebook laden wir eine PDF, erzeugen Embeddings mit `SentenceTransformerEmbeddings` und stellen Fragen zur PDF mit einem einfachen lokalen LLM Wrapper.

In [7]:
# Installiere nötige Pakete (einmalig)
!pip install langchain pypdf sentence-transformers faiss-cpu requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 4.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 5.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 4.1 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.8 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 4.6 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
# Imports
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
import requests

In [9]:
# Einfacher Wrapper für dein lokales LLM im Docker-Container (llm)
class LocalLlamaLLM(LLM):
    def __init__(self, endpoint: str = "http://llm:5000/completion"):
        self.endpoint = endpoint

    @property
    def _llm_type(self) -> str:
        return "local_llama"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        json_data = {"prompt": prompt, "n_predict": 200}
        response = requests.post(self.endpoint, json=json_data)
        if response.status_code == 200:
            return response.json()["content"]
        else:
            raise RuntimeError(f"LLM Anfrage fehlgeschlagen: {response.status_code} - {response.text}")

In [10]:
# Lade PDF und teile in Dokumente
pdf_path = "example.pdf"  # Pfad zur PDF-Datei
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Embeddings mit SentenceTransformer
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Erstelle FAISS Vektor-Datenbank
vectorstore = FAISS.from_documents(documents, embeddings)


/tmp/ipykernel_378/1313494915.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Setup Retrieval-QA Chain mit LocalLlamaLLM
llm = LocalLlamaLLM()
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

ValueError: "LocalLlamaLLM" object has no field "endpoint"

In [ ]:
# Beispiel-Frage an die PDF
frage = "Was ist das Thema der PDF?"
antwort = qa_chain.run(frage)
print("Frage:", frage)
print("Antwort:", antwort)